In [91]:
import requests
import json
import time
import pickle
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
# import censusdata
from sklearn.metrics.pairwise import haversine_distances
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import math
from bs4 import BeautifulSoup as BS
import ast
from itertools import combinations
from collections import Counter
#from scipy.cluster.hierarchy 
from pyproj import Geod
from shapely.geometry import Point, LineString

%matplotlib inline

In [92]:
hidden = pd.read_pickle('../data/API_key.pkl')
api_key = hidden.loc[0][1]

In [93]:
davidson_pred_black = gpd.read_file('../data/davidson_race_by_tract/davidson_tracts_pred_black.geojson')
davidson_pred_black.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geoid       25 non-null     object  
 1   name        25 non-null     object  
 2   total_pop   25 non-null     float64 
 3   total_pop_  25 non-null     float64 
 4   white_alon  25 non-null     float64 
 5   white_al_1  25 non-null     float64 
 6   black_afri  25 non-null     float64 
 7   black_af_1  25 non-null     float64 
 8   nativeamer  25 non-null     float64 
 9   nativeam_1  25 non-null     float64 
 10  asian_alon  25 non-null     float64 
 11  asian_al_1  25 non-null     float64 
 12  hawaiian_p  25 non-null     float64 
 13  hawaiian_1  25 non-null     float64 
 14  other_alon  25 non-null     float64 
 15  other_al_1  25 non-null     float64 
 16  two_or_mor  25 non-null     float64 
 17  two_or_m_1  25 non-null     float64 
 18  two_includ  25 non-null     float64 
 19  tw

In [210]:
all_types = ['gas_station',
             'supermarket',
             'grocery_store',
             'hospital',
             'clothing_store',
             'restaurant',
             'store',
             'night_club',
             'bar'] 

In [216]:
# Modified from https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

def search_places_by_coordinate(location, radius, types, api_key, sleep_sec = 2):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places = []
    params = {
        'location': location,
        'radius': radius,
        'types': types,
        'key': api_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  res.json()
    places.extend(results['results'])
    time.sleep(sleep_sec)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = res.json()
        places.extend(results['results'])
        time.sleep(sleep_sec)
    return places

## Because the Google API only lets you search for 1 type at a time, I've used a for loop to iterate through each type I need and create a list of those lists (nested lists, yay!)

In [281]:
ct10105 = []
ct10105_list = []
latitude10105 = []
longitude10105 = []
name10105 = []
type10105 = []

for i in all_types:
    ct10105_res = search_places_by_coordinate(location_10105, radius_10105, i, api_key)
    ct10105.append(ct10105_res)
    for list in ct10105:
        for element in list:
            latitude10105.append(element['geometry']['location']['lat'])
            longitude10105.append(element['geometry']['location']['lng'])
            name10105.append(element['name'])
            type10105.append(element['types'][0])

In [282]:
len(ct10105) # this should be 9 because we have 9 business types

9

In [283]:
len(ct10105[0]) # This would be the number of gas stations since that is the first type we iterated through

7

In [274]:
print(ct10105)

[[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 36.2520693, 'lng': -86.8297597}, 'viewport': {'northeast': {'lat': 36.2534926302915, 'lng': -86.8284251697085}, 'southwest': {'lat': 36.2507946697085, 'lng': -86.8311231302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/gas_station-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gas_pinlet', 'name': 'Shell', 'photos': [{'height': 4128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117265797567492229445">Rooster Davis</a>'], 'photo_reference': 'Aap_uEANVrb9YV2AhfiBYXJwOQMxWm_HgLtxPRRVHo4EJ9xKOstf-wUnGzOmT0QgE_2JyvxlaMgf2gn73vf54QT5P7WLMq1yIekTvptjbmNKez4DnzACD4odaGOxaE7dQKcQ7kFKbXSWbk9htDwWgddqlSVVCR2gqXdNdc4hvUJA48-3Qo15', 'width': 3096}], 'place_id': 'ChIJBQHTXnBeZIgR4BxNPRfisdI', 'plus_code': {'compound_code': '752C+R3 Nashville, TN, USA', 'global_code': '868M752C+R3'}, 'price_level': 2, 'rat

In [276]:
print(ct10105[0][0]['geometry']['location']['lat'])

36.2520693


In [277]:
ct10105_res = []
latitude10105 = []
longitude10105 = []
name10105 = []
type10105 = []

for list in ct10105:
    for element in list:
        latitude10105.append(element['geometry']['location']['lat'])
        longitude10105.append(element['geometry']['location']['lng'])
        name10105.append(element['name'])
        type10105.append(element['types'][0])
        

In [278]:
name10105

['Shell',
 "Cenex KCK's",
 'Kroger Fuel Center',
 "Brenda's Corner Market",
 'Kwik Sak',
 'Citgo',
 'Friendly Corner Market',
 'Family Dollar',
 "Tony's Foodland",
 'Nashville',
 'Fontanel',
 'Carl Black Chevy Woods Amphitheater',
 'Whites Creek High School',
 'Cedarwood Weddings',
 'Bates Nursery and Garden Center',
 'Dollar General',
 'Beaman Park Nature Center',
 'Nashville Public Library Bordeaux Branch',
 'Subway',
 "Flowers' Farms",
 'UPS Customer Center',
 'New Life Apostolic Church',
 'JR Construction Tennessee INC.',
 'Rebuild America With Music',
 'Realm3 Web Applications',
 'GES Inc',
 'SEE FOAM Insulation',
 'Tanksley Tree Service LLC',
 "Talbot's Corner",
 '2PhoneFree',
 'Further Farms',
 'Shell',
 'Imaginary Records',
 'Music City Zips',
 'Adventureworks Zipline Forest',
 'Pro Care Consulting',
 'The Inn At Fontanel',
 'JT Equity Group, LLC',
 'United States Postal Service',
 'First Horizon Bank',
 'All Professional Construction',
 'Three Generations Home Healthcare',
 'B

In [191]:
df10105 = pd.DataFrame(list(zip(name10105, type10105, latitude10105, longitude10105)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [192]:
df10105['tract'] = 'census_tract_10105'
df10105

,name,type,latitude,longitude,tract
0,Family Dollar,supermarket,36.211460,-86.839407,census_tract_10105
1,Tony's Foodland,grocery_or_supermarket,36.280344,-86.876424,census_tract_10105


In [180]:
test_map = folium.Map([36.24885, -86.85388])
for row_index, row_values in df10105.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    tool = [str(row_values['name']), str(row_values['type'])]
    
    marker = folium.Marker(
        location = loc, 
        tooltip = tool) 

    marker.add_to(test_map)
test_map

In [100]:
location_10106 = '36.25371, -86.80361'
radius_10106 = '4000'

In [161]:
ct10106 = search_places_by_coordinate(location_10106, radius_10106, all_types, api_key)
len(ct10106)

8

In [231]:
type(ct10106)

list

In [113]:
test_dict = {location_10105:radius_10105, location_10106:radius_10106}

In [119]:
test_name = []
test_type = []

for key, value in test_dict.items():
    results_10105_10106 =  search_places_by_coordinate(key, value, all_types, api_key)
    for i in results_10105_10106:
        test_name.append(i['name'])
        test_type.append(i['types'][0])


In [123]:
print(test_type)

['gas_station', 'gas_station', 'gas_station', 'convenience_store', 'gas_station', 'convenience_store', 'gas_station', 'convenience_store', 'gas_station', 'gas_station', 'convenience_store', 'gas_station', 'convenience_store', 'convenience_store']


In [117]:
len(results_10105_10106)

8

# Now that I've created the DataFrame for the first census tract, I will do the same for the others

In [232]:
latitude10106 = []
longitude10106 = []
name10106 = []
type10106 = []

for i in ct10106:
    latitude10106.append(i['geometry']['location']['lat'])
    longitude10106.append(i['geometry']['location']['lng'])
    name10106.append(i['name'])
    type10106.append(i['types'][0])

In [233]:
name10106

['Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart']

In [140]:
df10106 = pd.DataFrame(list(zip(name10106, type10106, latitude10106, longitude10106)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [141]:
df10106['tract'] = 'census_tract_10106'
df10106

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.252069,-86.829760,census_tract_10106
1,Tri-Star Energy,convenience_store,36.276629,-86.795924,census_tract_10106
2,Shell,gas_station,36.277810,-86.796850,census_tract_10106
3,Citgo,gas_station,36.232554,-86.781664,census_tract_10106
4,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10106
5,Marathon,gas_station,36.276661,-86.795952,census_tract_10106
6,Little Barn Market,convenience_store,36.231354,-86.781349,census_tract_10106
7,A a Mart,convenience_store,36.218555,-86.804160,census_tract_10106


In [142]:
location_10903 = '36.24932, -86.76382'
radius_10903 = '2716'

In [162]:
ct10903 = search_places_by_coordinate(location_10903, radius_10903, all_types, api_key)
len(ct10903)

11

In [163]:
latitude10903 = []
longitude10903 = []
name10903 = []
type10903 = []

for i in ct10903:
    latitude10903.append(i['geometry']['location']['lat'])
    longitude10903.append(i['geometry']['location']['lng'])
    name10903.append(i['name'])
    type10903.append(i['types'][0])

In [164]:
name10903

['Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market']

In [165]:
df10903 = pd.DataFrame(list(zip(name10903, type10903, latitude10903, longitude10903)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [166]:
df10903['tract'] = 'census_tract_10903'
df10903

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.262900,-86.757836,census_tract_10903
1,MAPCO,gas_station,36.229043,-86.758997,census_tract_10903
2,Shell,gas_station,36.268857,-86.755242,census_tract_10903
3,Citgo,gas_station,36.232554,-86.781664,census_tract_10903
4,Shell,gas_station,36.242914,-86.736170,census_tract_10903
5,Murphy Express,gas_station,36.243486,-86.757311,census_tract_10903
6,Exxon,gas_station,36.232835,-86.759098,census_tract_10903
7,Saint George Market,gas_station,36.231935,-86.758394,census_tract_10903
8,Dickerson Road Citgo,gas_station,36.229185,-86.759641,census_tract_10903
9,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10903


In [167]:
location_10904 = '36.23431, -86.77321'
radius_10904 = '2203'

In [168]:
ct10904 = search_places_by_coordinate(location_10904, radius_10904, all_types, api_key)
len(ct10904)

10

In [169]:
latitude10904 = []
longitude10904 = []
name10904 = []
type10904 = []

for i in ct10904:
    latitude10904.append(i['geometry']['location']['lat'])
    longitude10904.append(i['geometry']['location']['lng'])
    name10904.append(i['name'])
    type10904.append(i['types'][0])

In [170]:
name10904

['MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart']

In [181]:
df10904 = pd.DataFrame(list(zip(name10904, type10904, latitude10904, longitude10904)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [182]:
df10904['tract'] = 'census_tract_10904'
df10904

,name,type,latitude,longitude,tract
0,MAPCO,gas_station,36.229043,-86.758997,census_tract_10904
1,Citgo,gas_station,36.232554,-86.781664,census_tract_10904
2,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_10904
3,Murphy Express,gas_station,36.243486,-86.757311,census_tract_10904
4,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10904
5,Little Barn Market,convenience_store,36.231354,-86.781349,census_tract_10904
6,Exxon,gas_station,36.232835,-86.759098,census_tract_10904
7,Dickerson Road Citgo,gas_station,36.229185,-86.759641,census_tract_10904
8,Saint George Market,gas_station,36.231935,-86.758394,census_tract_10904
9,G dragon mart,gas_station,36.216211,-86.763359,census_tract_10904


In [ ]:
location_11001 = '36.23431, -86.77321'
radius_11001 = '2203'